<a href="https://colab.research.google.com/github/Gman80/nlp-aprendizaje/blob/main/Lab_HuggingFace_Tutor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from huggingface_hub import notebook_login

# Run this cell and follow the instructions to log in to Hugging Face Hub
# It's recommended to save your token as a Colab Secret named 'HF_TOKEN'
notebook_login()

print("Hugging Face login process initiated. Check the output above for status.")

Hugging Face login process initiated. Check the output above for status.


In [2]:
import torch
import transformers

print(f"Torch version: {torch.__version__}")
print(f"Transformers version: {transformers.__version__}")

Torch version: 2.8.0+cu126
Transformers version: 4.56.1


In [3]:
try:
    from datasets import load_dataset
    # Specify 'csv' as the dataset type and provide separator as '\t' for TSV
    dataset = load_dataset("csv", data_files="/content/drive/MyDrive/Colab Notebooks/Restaurant_Reviews.tsv", sep='\t')
    print("Dataset loaded successfully!")
    print(dataset)

except Exception as e:
    print(f"Error loading dataset: {e}")

Generating train split: 0 examples [00:00, ? examples/s]

Dataset loaded successfully!
DatasetDict({
    train: Dataset({
        features: ['Review', 'Liked'],
        num_rows: 1000
    })
})


In [4]:
from transformers import AutoTokenizer
from datasets import DatasetDict

# Load a tokenizer suitable for the task (e.g., for a sentiment analysis model)
# You might need to specify a model name here, depending on your downstream task
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased") # Example: using a BERT tokenizer

def tokenize_fn(example):
    return tokenizer(example["Review"], truncation=True, padding="max_length")

# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_fn, batched=True)

# Split the tokenized dataset into training and testing sets
train_test_split = tokenized_dataset["train"].train_test_split(test_size=0.2) # Using the 'train' split to split

# Rename the splits to 'train' and 'test' to match the Trainer's expectations
tokenized = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})

# Rename the 'Liked' column to 'labels' for the Trainer
tokenized = tokenized.rename_column("Liked", "labels")

print("Dataset tokenized, split, and labels column renamed successfully!")
print(tokenized)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset tokenized, split, and labels column renamed successfully!
DatasetDict({
    train: Dataset({
        features: ['Review', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 800
    })
    test: Dataset({
        features: ['Review', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 200
    })
})


In [5]:
import torch

# Check if CUDA is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the model and move it to the selected device
from transformers import AutoModelForSequenceClassification

model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)

print(f"Model loaded successfully and moved to {device}.")

Using device: cuda


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded successfully and moved to cuda.


In [6]:
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification

model_name = "distilbert-base-uncased"
# Assuming 'device' is defined in a previous cell
# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch", # Corrected argument name
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01
)

trainer = Trainer(
    model=model, # Assuming 'model' is defined in a previous cell
    args=training_args,
    train_dataset=tokenized["train"], # Assuming 'tokenized' is your dataset dictionary
    eval_dataset=tokenized["test"]   # Assuming 'tokenized' is your dataset dictionary
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mercadogrmo (mercadogrmo-kaggle) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,0.266113
2,No log,0.212110
3,No log,0.217107


TrainOutput(global_step=150, training_loss=0.2721435038248698, metrics={'train_runtime': 140.4186, 'train_samples_per_second': 17.092, 'train_steps_per_second': 1.068, 'total_flos': 317921756774400.0, 'train_loss': 0.2721435038248698, 'epoch': 3.0})